In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import tensorflow as tf
import tensorflow_io as tfio
import os
from tqdm import tqdm
import cv2
from PIL import Image
from joblib import Parallel, delayed
import gc
from PIL import Image as im
import random

In [2]:
import sys
!{sys.executable} -m pip install dicomsdl
#!pip install /kaggle/input/rsnapacks/dicomsdl-0.109.1-cp36-cp36m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#import pydicom as dicom
import dicomsdl as dicom
import tensorflow as tf
import tensorflow_io as tfio
from PIL import Image as im
import os
from tqdm import tqdm
import cv2
from PIL import Image
from joblib import Parallel, delayed
import gc
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.7 MB/s eta 0:00:00


In [3]:
datasetPath = '/kaggle/input/rsna-breast-cancer-detection/train.csv'
imgPath = '/kaggle/input/rsna-breast-cancer-detection/train_images/'

def crop(sideName, imgName):
    if sideName == 'L':
        colind=[]
        for r,row in enumerate(imgName):
            for c,col in enumerate(row):
                if col==0:
                    colind.append(c)
                    break
        crop_size = max(colind)
        imgName = imgName[0:512,0:crop_size]
        imgName = cv2.resize(imgName,(128,128))
        
    if sideName == 'R':
        colind=[]
        for r,row in enumerate(imgName):
            for c,col in enumerate(row):
                if col!=0:
                    colind.append(c)
                    break
        crop_size = min(colind)
        imgName = imgName[0:512,crop_size:512]
        imgName = cv2.resize(imgName,(128,128))
    
    return imgName    

    gc.collect()
    
def crop_reverse(sideName, imgName):
    if sideName == 'R':
        colind=[]
        for r,row in enumerate(imgName):
            for c,col in enumerate(row):
                if col==0:
                    colind.append(c)
                    break
        crop_size = max(colind)
        imgName = imgName[0:512,0:crop_size]
        imgName = cv2.resize(imgName,(128,128))
        
    if sideName == 'L':
        colind=[]
        for r,row in enumerate(imgName):
            for c,col in enumerate(row):
                if col!=0:
                    colind.append(c)
                    break
        crop_size = min(colind)
        imgName = imgName[0:512,crop_size:512]
        imgName = cv2.resize(imgName,(128,128))
    
    return imgName    

    gc.collect()
    

def img_process(i,filename,sides):
    
    #ds = dicom.dcmread(filename)
    ds = dicom.open(filename)
    ds = ds.pixelData()
    ds = cv2.normalize(ds, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    ds = cv2.resize(ds,(512,512))
    ds = np.where(ds >= 0.9999, 0,ds)
    
    try:
        ds = np.array(crop(sides[i], ds))   
    except:
        ds = np.array(crop_reverse(sides[i], ds))
    return ds 

    #train_data.loc[i,'img_data'] = [img_fin]
    #train_data.to_csv('/kaggle/working/training_img_data.csv') 
    
    gc.collect()

def dcmToPix(datasetPath, imgPath):
    dataset = pd.read_csv(datasetPath)
    dataset=dataset[:50]
    
    
    patient_ids = dataset['patient_id']
    image_ids = dataset['image_id']
    sides  = dataset['laterality']

    imgData = []

    for pi, ii, leng in zip(patient_ids, image_ids, range(len(patient_ids))):
        imgData.append(imgPath + str(pi) + '/' + str(ii) + '.dcm')

    dataset['img_data'] = " "
    
    result = Parallel(n_jobs=128)(\
    delayed(img_process)(i, fname, sides) for i, fname in zip(range(len(imgData)),tqdm(imgData))\
    )
    
    dataset['img_data'] = result
    dataset.to_pickle('imgData.pkl' )
    
    return result
    

In [4]:
with (open('/kaggle/input/output/imgData.pkl', "rb")) as openfile:
     imgData = pickle.load(openfile)

In [5]:
#datacancer = pd.read_csv(datasetPath)
#imgDataFrame = {'cancer':datacancer['cancer'][:50], 'img_data':imgData}
#imgData2 = pd.DataFrame(imgDataFrame)

#imgData=imgData2

In [6]:
imgdata_pos = imgData[imgData['cancer'] == 1]
imgdata_neg = imgData[imgData['cancer'] == 0]

imgdata_pos = imgdata_pos.sample(frac = 1)
imgdata_neg = imgdata_neg.sample(frac = 1)

imgdata_neg = imgdata_neg.sample(frac= 1)

frames = 3*[imgdata_pos]
frames.append(imgdata_neg)
imgdata_shuff = pd.concat(frames)
imgdata_shuff = imgdata_shuff.sample(frac=1)

In [7]:
print(imgdata_shuff['cancer'].value_counts())

0    53548
1     3474
Name: cancer, dtype: int64


In [8]:
def random_rotate(imgData):
    thresh = 0.15
    
    imgData = imgData.reshape(128,128)
    imgData = np.clip(imgData,thresh,1)
    imgData = imgData[25:105,25:105]
    
    imgData = im.fromarray(imgData)
    imgData = np.asarray(imgData.rotate(random.randint(0, 5)))
    return imgData

In [9]:
imgDataList=[]
for j in tqdm(imgdata_shuff['img_data']):
    imgDataList.append(random_rotate(j))
imgdata_shuff['img_data'] = imgDataList

100%|██████████| 57022/57022 [00:11<00:00, 4966.05it/s]


In [10]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(imgdata_shuff, test_size=0.3, random_state=42, shuffle=True)


train_target = np.array(train['cancer'])
train_features=[]
for i in train['img_data']:
    train_features.append([i])
train_features=np.array(train_features)



featureTransform = train_features.reshape(len(train_features), 6400)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
featureTransform = scaler.fit_transform(featureTransform)
#norm_features= []
#for i in range(len(train_features)):
#    norm_features.append(scaler.fit_transform(train_features[i][0]))
#train_features=np.array(norm_features)


backTransform = featureTransform.reshape(len(train_features),80,80)
train_features = backTransform.reshape(len(train_features),80,80,1)



In [11]:
from keras.models import Sequential
from keras.layers import LeakyReLU
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D, Dropout, GlobalMaxPooling2D
model = Sequential()
model.add(Conv2D(64, 3, activation = "relu", input_shape = (80,80,1)))
model.add(MaxPooling2D())
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.3))
model.add(Conv2D(32, 3, activation = "relu"))
model.add(MaxPooling2D())
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(16, 3, activation = "relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation = 'relu')) 
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))




from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives )
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives )
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))





#loss = tf.keras.losses.BinaryFocalCrossentropy(\
#    apply_class_balancing=True, gamma=5, from_logits=True,\
#    reduction=tf.keras.losses.Reduction.NONE)


model.compile(optimizer =tf.keras.optimizers.Adam(learning_rate=0.0001),\
              loss= 'binary_crossentropy', metrics=['accuracy'])



2023-01-24 21:41:12.680623: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [12]:
#from tensorflow import keras
#model = keras.models.load_model("/kaggle/input/pre-trained-model-of-breast-cancer/trained_model_breast_cancer3.h5")


In [13]:
weights = {0:1, 1:15}
model.fit(train_features, train_target,  class_weight=weights,validation_split=0.3,batch_size = 64,epochs=200)




2023-01-24 21:41:13.562287: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200
437/437 [==============================] - 178s 404ms/step - loss: 1.2892 - accuracy: 0.5274 - val_loss: 0.6843 - val_accuracy: 0.4847
Epoch 2/200
437/437 [==============================] - 171s 392ms/step - loss: 1.2782 - accuracy: 0.4884 - val_loss: 0.6912 - val_accuracy: 0.3607
Epoch 3/200
437/437 [==============================] - 168s 383ms/step - loss: 1.2750 - accuracy: 0.4676 - val_loss: 0.7228 - val_accuracy: 0.2778
Epoch 4/200
437/437 [==============================] - 168s 384ms/step - loss: 1.2715 - accuracy: 0.4858 - val_loss: 0.6942 - val_accuracy: 0.4257
Epoch 5/200
437/437 [==============================] - 165s 378ms/step - loss: 1.2660 - accuracy: 0.5057 - val_loss: 0.7103 - val_accuracy: 0.4104
Epoch 6/200
437/437 [==============================] - 164s 376ms/step - loss: 1.2616 - accuracy: 0.5550 - val_loss: 0.6898 - val_accuracy: 0.5602
Epoch 7/200
437/437 [==============================] - 167s 381ms/step - loss: 1.2564 - accuracy: 0.5751 - val_loss: 0

In [14]:
model.save("/kaggle/working/trained_model_breast_cancer3.h5")

In [15]:
#from tensorflow import keras
#savedModel = keras.models.load_model("/kaggle/input/pre-trained-model-of-breast-cancer/trained_model_breast_cancer3.h5")



In [16]:
test_features=[]
for i in test['img_data']:
    test_features.append([i])
test_features=np.array(test_features)
 

    
featureTransform = test_features.reshape(len(test_features), 6400)
scaler = StandardScaler()
featureTransform =scaler.fit_transform(featureTransform)
#norm_features= []
#for i in range(len(test_features)):
#    norm_features.append(scaler.fit_transform(test_features[i][0]))
#test_features=np.array(norm_features)


backTransform = featureTransform.reshape(len(test_features),80,80)
test_features = backTransform.reshape(len(test_features),80,80,1)



test_target = np.array(test['cancer'])

In [17]:
pred = model.predict(test_features)
bin_pred = []
for i in pred:
    if i>=0.5:
        bin_pred.append(1)
    else:
        bin_pred.append(0)
bin_pred = np.array(bin_pred)


In [18]:
import sklearn
acc = sklearn.metrics.accuracy_score(test_target, bin_pred)
print(acc)

countzero=0
countone=0
countzerot=0
countonet=0
for i, j in zip(test_target, bin_pred):
    if i==0 and j==0:
        countzero+=1
    if i==1 and j==1:
        countone+=1
    if i==1:
        countonet+=1
    if i==0:
        countzerot+=1
print(countzero/countzerot,countone/countonet)

0.9476822353422576
0.960296222540295 0.7524084778420038


In [19]:

def recall_m(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives )
    return recall

def precision_m(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives )
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))


print(f1_m(test_target,bin_pred))

0.6357346357346357


In [20]:
datasetPath = '/kaggle/input/rsna-breast-cancer-detection/test.csv'
imgPath = '/kaggle/input/rsna-breast-cancer-detection/test_images/'

testImgData = dcmToPix(datasetPath, imgPath)

 75%|███████▌  | 3/4 [00:00<00:00,  8.33it/s]


In [21]:
datacancer = pd.read_csv(datasetPath)
imgDataFrame = {'img_data':testImgData}
imgData2 = pd.DataFrame(imgDataFrame)
testImgData=imgData2
imgdata_shuff = testImgData.sample(frac=1)

In [22]:
imgDataList=[]
for j in tqdm(imgdata_shuff['img_data']):
    imgDataList.append(random_rotate(j))
imgdata_shuff['img_data'] = imgDataList

100%|██████████| 4/4 [00:00<00:00, 2008.77it/s]


In [23]:
test=imgdata_shuff


test_features=[]
for i in test['img_data']:
    test_features.append([i])
test_features=np.array(test_features)


featureTransform = test_features.reshape(len(test_features), 6400)
scaler = StandardScaler()
featureTransform =scaler.fit_transform(featureTransform)
#norm_features= []
#for i in range(len(test_features)):
#    norm_features.append(scaler.fit_transform(test_features[i][0]))
#test_features=np.array(norm_features)


backTransform = featureTransform.reshape(len(test_features),80,80)
test_features = backTransform.reshape(len(test_features),80,80,1)




In [24]:
pred = model.predict(test_features)
bin_pred = []
for i in pred:
    if i>=0.5:
        bin_pred.append(1)
    else:
        bin_pred.append(0)
bin_pred = np.array(bin_pred)
print(bin_pred)

[0 0 0 0]


In [25]:
testData = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')

submissionFrame={'prediction_id':testData['prediction_id'],'cancer':bin_pred}
submission = pd.DataFrame(submissionFrame)
print(submission)

  prediction_id  cancer
0       10008_L       0
1       10008_L       0
2       10008_R       0
3       10008_R       0
